### Librerías

In [1]:
import numpy as np
import warnings
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tensorflow.keras.models import Model, Sequential 
from tensorflow.keras.layers import Input, Dense, Conv2D, Dropout, BatchNormalization, Conv2DTranspose, UpSampling2D, Flatten, Activation, Reshape
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import RMSprop, Adam, Adadelta 
from google.colab import drive
%matplotlib inline

In [2]:
warnings.filterwarnings("ignore", category=FutureWarning)

Acceder a los datos de Google Drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


Cargar datos y establecer (n_muestras, 224, 224, 1) como "shape".

In [4]:
images = "/content/drive/MyDrive/mamma.npy"
np_images = np.load(images, allow_pickle=True)
normalized_images = np_images/255
normalized_images = np.reshape(normalized_images,(normalized_images.shape[0],224,224,1))

### Generador

In [ ]:
def generator_architecture(depth=64, p=0.4):
    noise_shape = Input((32,))

    dense1 = Dense(7*7*depth)(noise_shape)
    dense1 = BatchNormalization(momentum=0.9)(dense1) 
    dense1 = Activation(activation='relu')(dense1)
    dense1 = Reshape((7,7,depth))(dense1)
    dense1 = Dropout(p)(dense1)
    
    conv1 = UpSampling2D()(dense1)
    conv1 = Conv2DTranspose(int(depth/2), 
                            kernel_size=5, padding='same', 
                            activation=None,)(conv1)
    conv1 = BatchNormalization(momentum=0.9)(conv1)
    conv1 = Activation(activation='relu')(conv1)
    
    conv2 = UpSampling2D()(conv1)
    conv2 = Conv2DTranspose(int(depth/4), 
                            kernel_size=5, padding='same', 
                            activation=None,)(conv2)
    conv2 = BatchNormalization(momentum=0.9)(conv2)
    conv2 = Activation(activation='relu')(conv2)
    
    conv3 = UpSampling2D()(conv2)
    conv3 = Conv2DTranspose(int(depth/8), 
                            kernel_size=5, padding='same', 
                            activation=None,)(conv3)
    conv3 = BatchNormalization(momentum=0.9)(conv3)
    conv3 = Activation(activation='relu')(conv3)
    
    conv4 = UpSampling2D()(conv3)
    conv4 = Conv2DTranspose(int(depth/16), 
                            kernel_size=5, padding='same', 
                            activation=None,)(conv4)
    conv4 = BatchNormalization(momentum=0.9)(conv4)
    conv4 = Activation(activation='relu')(conv4)
    
    conv5 = UpSampling2D()(conv4)
    conv5 = Conv2DTranspose(int(depth/16), 
                            kernel_size=5, padding='same', 
                            activation=None,)(conv5)
    conv5 = BatchNormalization(momentum=0.9)(conv5)
    conv5 = Activation(activation='relu')(conv5)

    image = Conv2DTranspose(1, kernel_size=5, padding='same', 
                   activation='tanh')(conv5)
 
    model = Model(inputs=noise_shape, outputs=image)
    
    return model

In [ ]:
generator = generator_architecture()
generator.summary()

In [ ]:
def discriminator_architecture(depth=64, p=0.4):

    image = Input((224,224,1))

    conv1 = Conv2D(depth*1, 5, strides=2, 
                   padding='same', activation='relu')(image)
    conv1 = BatchNormalization(momentum=0.9)(conv1)
    conv1 = LeakyReLU(alpha=0.2)(conv1)
    conv1 = Dropout(p)(conv1)
    
    conv2 = Conv2D(depth*2, 5, strides=2, 
                   padding='same', activation='relu')(conv1)
    conv2 = LeakyReLU(alpha=0.2)(conv2)             
    conv2 = BatchNormalization(momentum=0.9)(conv2)               
    conv2 = Dropout(p)(conv2)
    
    conv3 = Conv2D(depth*4, 5, strides=2, 
                   padding='same', activation='relu')(conv2)
    conv3 = LeakyReLU(alpha=0.2)(conv3)
    conv3 = BatchNormalization(momentum=0.9)(conv3)  
    conv3 = Dropout(p)(conv3)
    
    conv4 = Conv2D(depth*8, 5, strides=2, 
                   padding='same', activation='relu')(conv3)
    conv4 = LeakyReLU(alpha=0.2)(conv4)               
    conv4 = BatchNormalization(momentum=0.9)(conv4)                 
    conv4 = Flatten()(Dropout(p)(conv4))

    prediction = Dense(1, activation='sigmoid')(conv4)

    model = Model(inputs=image, outputs=prediction)
    
    return model

In [ ]:
discriminator = discriminator_architecture()
discriminator.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 224, 224, 1)]     0         
                                                                 
 conv2d_9 (Conv2D)           (None, 112, 112, 64)      1664      
                                                                 
 batch_normalization_32 (Bat  (None, 112, 112, 64)     256       
 chNormalization)                                                
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 112, 112, 64)      0         
                                                                 
 dropout_12 (Dropout)        (None, 112, 112, 64)      0         
                                                                 
 conv2d_10 (Conv2D)          (None, 56, 56, 128)       204928    
                                                           

In [ ]:
discriminator.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.0004, decay=6e-8, clipvalue=1.0), metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
noise = Input(shape=(32,))
synthetic_img = generator(noise)

In [ ]:
discriminator.trainable = False

In [ ]:
discriminator_pred = discriminator(synthetic_img)
adversarial_model = Model(noise, discriminator_pred)

In [ ]:
adversarial_model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.0004, decay=3e-8, clipvalue=1.0), metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Guardar modelo e imágenes creadas por cada "x" epochs que se establezcan.

In [ ]:
GENERATOR_PATH = "/content/drive/MyDrive/generator_model_DCGAN/"
IMAGES = "/content/drive/MyDrive/images/"

In [ ]:
def training_procedure(epochs=4000, batch=32):
    
    discriminator_stats, adversarial_stats = [], []
    discriminator_loss , discriminator_acc = 0, 0
    adversarial_loss, adversarial_acc = 0, 0

    for epoch in range(1, epochs+1):
        real_imgs = np.reshape(normalized_images[np.random.choice(normalized_images.shape[0],batch,replace=False)],(batch,224,224,1))
        fake_imgs = generator.predict(np.random.uniform(-1.0, 1.0, size=[batch, 32]))
    
        real_and_fake_imgs = np.concatenate((real_imgs,fake_imgs))
        
        discriminator_labels = np.ones([2*batch,1])
        discriminator_labels[batch:,:] = 0
        
        discriminator_stats.append(discriminator.train_on_batch(real_and_fake_imgs,discriminator_labels))
        discriminator_loss += discriminator_stats[-1][0]
        discriminator_acc += discriminator_stats[-1][1]
        
        noise = np.random.uniform(-1.0, 1.0, size=[batch, 32])
        y = np.ones([batch,1])

        adversarial_stats.append(adversarial_model.train_on_batch(noise,y)) 
        adversarial_loss += adversarial_stats[-1][0]
        adversarial_acc += adversarial_stats[-1][1]
             
        print("%d: [D loss: %f, acc: %f]" % (epoch, discriminator_loss/epoch, discriminator_acc/epoch))

        if (epoch+1)%100 == 0: 

          noise = np.random.uniform(-1.0, 1.0, size=[16, 32])
          gen_imgs = generator.predict(noise)
          plt.figure(figsize=(7,7))

          for k in range(gen_imgs.shape[0]):
            plt.subplot(4, 4, k+1)
            plt.imshow(gen_imgs[k, :, :, 0], cmap='gray')
            plt.axis('off')
            plt.savefig(IMAGES+'epoch%d.png' % (epoch))
 
          plt.tight_layout()
          plt.show()

    generator.save(GENERATOR_PATH+'gan_generator_%d.h5' % (epoch))
    
    return discriminator_stats

In [ ]:
discriminator_metrics, adversarial_metrics = training_procedure()